In [1]:
import pandas as pd

In [2]:
from glob import glob

In [3]:
def get_dataset(file_names, auth):
    dataset = pd.read_csv(file_names[0], sep=" ").iloc[:,0:4]
    col_names = dataset.columns
    dataset = dataset.rename(columns={col_names[0]:'sample', col_names[1]: 'degree_x',
                           col_names[2]:'degree_y', col_names[3]: 'is_valid'})
    i = 0
    prob = []
    for file in file_names[1:]:
        i = i + 1
        sample = pd.read_csv(file, sep = " ").iloc[:,0:4]
        col_names = sample.columns
        sample = sample.rename(columns={col_names[0]:'sample', col_names[1]: 'degree_x',
                           col_names[2]:'degree_y', col_names[3]: 'is_valid'})
        dataset = pd.concat([dataset, sample])
        
        if sample.shape[1] != 4:
            prob = prob + [i]
        
    dataset['y'] = auth
    
    return dataset, prob

In [4]:
file_names = glob(r"C:/Users/anark/Downloads/ETPAD.v2/ETPAD.v2/LIVE EYE MOVEMENTS/*.txt",  recursive = True)
data1,problem = get_dataset(file_names, 0)

In [5]:
problem

[]

In [7]:
data1.to_csv('data1.csv')

In [19]:
file_names = glob(r"C:/Users/anark/Downloads/ETPAD.v2/ETPAD.v2/SAS_I EYE MOVEMENTS/*.txt",  recursive = True)
data2,problem2 = get_dataset(file_names, 1)

In [20]:
file_names = glob(r"C:/Users/anark/Downloads/ETPAD.v2/ETPAD.v2/SAS_II EYE MOVEMENTS/*.txt",  recursive = True)
data3,problem3 = get_dataset(file_names, 2)

In [21]:
data2.to_csv('data2.csv')
data3.to_csv('data3.csv')

In [22]:
dataset = pd.concat([data1, data2, data3])

In [24]:
dataset.to_csv('dataset.csv')

In [25]:
problem2

[]

In [26]:
problem3

[]